# MIA against a Simple Logistic Regression Algorithm

We will perform a Membership Inference Attacks against a self-made LR model, trained on well known iris dataset.

In [1]:
# import basic libraries
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import random
import sys

from sklearn.datasets import load_iris, load_breast_cancer, load_wine, fetch_olivetti_faces, fetch_lfw_pairs, load_diabetes
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from tqdm.notebook import tqdm

!pip install mia
from mia.estimators import ShadowModelBundle, AttackModelBundle, prepare_attack_data

  Created wheel for mia: filename=mia-0.1.2-py3-none-any.whl size=11106 sha256=671f35bc7e319981f4ff3c82e17da43798ca7a8e090c258ab714dd49fc35475d
  Stored in directory: /root/.cache/pip/wheels/11/cb/28/c0c2be5bebacd827e384b58ccfe4833ca3bffc1aa0086766d7
Successfully built mia


In [2]:
#load the data set
X, y = load_wine(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, shuffle=True, random_state=0)


In [3]:
# create the mode
model = DecisionTreeClassifier()

In [4]:
# fit the model and check accuracy scores
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f"Model Accuracy: {accuracy_score(y_test, y_pred)}") 

Model Accuracy: 0.9101123595505618


In [5]:
NUM_CLASSES = 2
SHADOW_DATASET_SIZE = 35
ATTACK_TEST_DATASET_SIZE = 4000

def target_model_fn():
  return DecisionTreeClassifier()

def attack_model_fn():
  return RandomForestClassifier()

smb = ShadowModelBundle(
        target_model_fn,
        shadow_dataset_size=SHADOW_DATASET_SIZE,
        num_models=50,

    )

X_shadow, y_shadow = smb.fit_transform(
        X_test,
        y_test,
        
    )

print(X_shadow.shape)

 # ShadowModelBundle returns data in the format suitable for the AttackModelBundle.
amb = AttackModelBundle(attack_model_fn, num_classes=NUM_CLASSES)

# Fit the attack models.
print("Training the attack models...")
amb.fit(X_shadow, y_shadow)

    # Test the success of the attack.

    # Prepare examples that were in the training, and out of the training.
data_in = X_train, y_train
data_out = X_test, y_test

    # Compile them into the expected format for the AttackModelBundle.
attack_test_data, real_membership_labels = prepare_attack_data(
        model, data_in, data_out
    )

    # Compute the attack accuracy.
attack_guesses = amb.predict(attack_test_data)
attack_accuracy = np.mean(attack_guesses == real_membership_labels)

print(attack_accuracy)

(3500, 4)
Training the attack models...
0.5449438202247191


## Membership Inference Attack

### Assumptions
- Each and every Shadow dataset is disjoint with target dataset

### Planning
- Create $n$ shadow models
- Create $n$ shadow datasets
- Train the shadow models
- Query each one of them and get a attack training-dataset
- Create and Train Attack Model
- Perform an attack in the victim model
- Evaluate the attack



In [6]:
N_SHADOW_MODELS = 50
SHADOW_DATASET_SIZE = X_train.shape[0]


In [7]:
## TODO IMPLEMENT SYNTESIS OF SHADOW DATA

# produce a random record
def rand_record(size, mean, std):
  v = np.random.normal(mean, std, size)
  return np.reshape(v, (1, -1))
#
def rand_record2(x, k_features):
  changed_features = set()
  for k in range(k_features):
    # pick a random index
    i = random.randint(0, x.size-1) 
    # don't change the same feature twice
    while i in changed_features:
      i = random.randint(0, x.size-1)

    # note the feature as changed
    changed_features.add(i)

    # change the feature by a random value
    x[0][i] = x[0][i] + np.random.randn() * np.random.normal(0, 4) 


def synthesize(model, dim, data_mean, data_std, c, k_max, k_min, iter_max, conf_min, rej_max):
  x = rand_record(dim, data_mean, data_std)
  y_conf_star = 0.0
  rej = 0;
  k = k_max

  for iter in range(iter_max):
    # predict outcome
    y = model.predict_proba(x)
    y_conf = y[0][c] # the confidence of class 'c' is the probability the classifier predicts
    
    # if the prediction confidence is high and the classifier predicts that x is of class 'c' 
    # then we flip a coin to decide if we gonna add this datapoint in our shadow training data  
    if y_conf > y_conf_star and c == np.argmax(y):
      # flip a coin in {0, 1} and sample
      if np.random.randint(2) > 0:
        # add the datapoint into synthetic dataset
        return x.copy()

      # update metrics
      y_conf_star = y_conf
      rej = 0

    else:
      #reject the datapoint
      rej += 1
      
      # too many rejections => decrease the features to modify and renew the counter
      if rej > rej_max:
        k = max(k_min, (k+1)//2)
        rej = 0

    x = rand_record2(x, k)

    return None

In [8]:
# create the shadow train and test datasets
shadow_training_sets = []
shadow_test_sets = []
values = range(SHADOW_DATASET_SIZE)
for i in range(N_SHADOW_MODELS):
  dataset = None
  with tqdm(total=len(values), file=sys.stdout) as pbar:
    pbar.set_description("Creating shadow-dataset: %d" %(1+i))
    c = 0
    for j in range(SHADOW_DATASET_SIZE):
      c = (c + 1)%(y.max() + 1)

      conf = 0.9
      x = synthesize(model, int(X.shape[1]), X.mean(), X.std(), c, int(X.shape[1]), 1, 100, conf, 25)
      while x is None:
        x = synthesize(model, int(X.shape[1]), X.mean(), X.std(), c, int(X.shape[1]), 1, 200, conf, 25)

      if x is not None:
        # if x is not none then add it in the dataset
        x = np.concatenate((x, [[c]]), axis=1)
        dataset = np.concatenate((dataset, x.copy())) if dataset is not None else x.copy()
    
      pbar.update(1)
    
  train, test = train_test_split(dataset.copy(), test_size=0.1, random_state=0, shuffle=True)
  train_x, train_y = train[:, :-1], train[:, -1]
  test_x, test_y = test[:, :-1], test[:, -1]
  shadow_training_sets.append((train_x,train_y))
  shadow_test_sets.append((test_x, test_y))


Creating shadow-dataset: 50: 100%|██████████| 89/89 [00:39<00:00,  2.27it/s]


In [9]:
#Now we will train all the shadow models

# init the shadow models
shadow_models = []

for i in range(N_SHADOW_MODELS):
  shadow_X_train, shadow_y_train = shadow_training_sets[i]
  shadow_model = DecisionTreeClassifier(random_state=0).fit(shadow_X_train, shadow_y_train)
  shadow_models.append(shadow_model)

In [10]:
# Evaluate the shadow models accuracy
for i in range(N_SHADOW_MODELS):
  shadow_X_test, shadow_y_test = shadow_test_sets[i]
  y_pred = shadow_models[i].predict(shadow_X_test)
  
  print(f"Shadow-Model-{i} Accuracy: {accuracy_score(shadow_y_test, y_pred)}")

Shadow-Model-0 Accuracy: 0.8888888888888888
Shadow-Model-1 Accuracy: 1.0
Shadow-Model-2 Accuracy: 1.0
Shadow-Model-3 Accuracy: 1.0
Shadow-Model-4 Accuracy: 0.5555555555555556
Shadow-Model-5 Accuracy: 1.0
Shadow-Model-6 Accuracy: 0.8888888888888888
Shadow-Model-7 Accuracy: 0.7777777777777778
Shadow-Model-8 Accuracy: 1.0
Shadow-Model-9 Accuracy: 0.8888888888888888
Shadow-Model-10 Accuracy: 0.7777777777777778
Shadow-Model-11 Accuracy: 0.8888888888888888
Shadow-Model-12 Accuracy: 0.8888888888888888
Shadow-Model-13 Accuracy: 0.8888888888888888
Shadow-Model-14 Accuracy: 0.8888888888888888
Shadow-Model-15 Accuracy: 1.0
Shadow-Model-16 Accuracy: 1.0
Shadow-Model-17 Accuracy: 1.0
Shadow-Model-18 Accuracy: 0.6666666666666666
Shadow-Model-19 Accuracy: 0.8888888888888888
Shadow-Model-20 Accuracy: 0.5555555555555556
Shadow-Model-21 Accuracy: 0.8888888888888888
Shadow-Model-22 Accuracy: 0.8888888888888888
Shadow-Model-23 Accuracy: 1.0
Shadow-Model-24 Accuracy: 0.8888888888888888
Shadow-Model-25 Accu

In [11]:
# Construct the attack dataset
# Labeling 'in' with 1 and 'out' with 0

D_attack = None 

for i in range(N_SHADOW_MODELS):
  # 'in' labeled data
  shadow_X_train, y_in = shadow_training_sets[i]
  y_prob_in = shadow_models[i].predict_proba(shadow_X_train)

  # 'out' labeled data
  shadow_X_test, y_out = shadow_test_sets[i]
  y_prob_out = shadow_models[i].predict_proba(shadow_X_test)

  in_labeled = np.concatenate((y_in.reshape(-1, 1), y_prob_in, model.predict(shadow_X_train).reshape(-1, 1), np.ones(shape=(y_in.shape[0], 1))), axis=1)
  out_labeled = np.concatenate((y_out.reshape(-1, 1), y_prob_out, model.predict(shadow_X_test).reshape(-1, 1), np.zeros(shape=(y_out.shape[0], 1))), axis=1)
  # create a attack dataset batch
  D_attack_i = np.concatenate((in_labeled, out_labeled)) 

  # append to the rest of the batches
  D_attack = np.concatenate((D_attack, D_attack_i), axis=0) if D_attack is not None else D_attack_i

np.random.shuffle(D_attack)

#D_attack instance format <c, proba_vec from shadow model_i, predicted class, 'in'/'out'>

In [12]:
attack_models = []
for c in range(y.max()+1):
  D = D_attack[D_attack[:, 0] == c]
  m = RandomForestClassifier().fit(D[:, 1:-1], D[:, -1])
  attack_models.append(m)

In [13]:
# Actual attack eval
# X format : <class, prob_vec from target, accuracy_score>
# all data are 'in' the target training set
in_labeled = np.concatenate((y_train.reshape(-1, 1), model.predict_proba(X_train), model.predict(X_train).reshape(-1, 1)), axis=1)
out_labeled = np.concatenate((y_test.reshape(-1, 1), model.predict_proba(X_test), model.predict(X_test).reshape(-1, 1)), axis=1)
X_attack_real = np.concatenate((in_labeled, out_labeled))

y_true = np.concatenate((np.ones((X_train.shape[0], 1)), np.zeros((X_test.shape[0], 1))))


for c in range(y.max()+1):
  X_targets = X_attack_real[X_attack_real[:, 0] == c, 1:] # get all the <prob-vecs> of predictions for class c
  y_attack_pred_real = attack_models[c].predict(X_targets)
  acc = accuracy_score(y_true[X_attack_real[:, 0] == c], y_attack_pred_real)
  print(f"({c})Real Attack accuracy: {acc}")



(0)Real Attack accuracy: 0.576271186440678
(1)Real Attack accuracy: 0.5211267605633803
(2)Real Attack accuracy: 0.5


In [14]:
in_labeled[in_labeled[:, -1] == 1].shape

(31, 5)

In [15]:
print(in_labeled[in_labeled[:, -1] == 0].shape)
print(in_labeled[in_labeled[:, -1] == 1].shape)
print(out_labeled[out_labeled[:, -1] == 0].shape)
print(out_labeled[out_labeled[:, -1] == 1].shape)


(34, 5)
(31, 5)
(26, 5)
(36, 5)
